In [1]:
import numpy as np 
import pandas as pd 
import json
import re
from tqdm.notebook import tqdm

In [2]:
train = pd.read_csv('/kaggle/input/title-generation/train.csv')
test = pd.read_csv('/kaggle/input/title-generation/test.csv')

In [3]:
train['id'] = train.index
test['id'] = test.index

In [4]:
train = np.array(train)
test = np.array(test)

In [5]:
#вспомагательная функция для поиска индекса, с которого начинается найденная строка
def find_all(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

In [6]:
#создаем директорию
!mkdir data

In [7]:
# Convert training data

output = {}
output['version'] = 'v1.0'
output['data'] = []

for line in train:
    paragraphs = []
    
    context = line[0]
    
    qas = []
    question = 'Title'   
    qid = line[2]
    answers = []
    answer = line[1]
    if type(answer) != str or type(context) != str or type(question) != str:
        print(context, type(context))
        print(answer, type(answer))
        print(question, type(question))
        continue
    answer_starts = find_all(context, answer)
    if len(answer_starts) > 0:
        
        for answer_start in answer_starts:
            answers.append({'answer_start': answer_start, 'text': answer})  
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
    
        paragraphs.append({'context': context, 'qas': qas})
        output['data'].append({'title': 'None', 'paragraphs': paragraphs})


with open('data/train.json', 'w') as outfile:
    json.dump(output, outfile)

In [8]:
output['data'][0]

{'title': 'None',
 'paragraphs': [{'context': 'we present the photometric calibration of the swift ultraviolet/optical telescope (uvot) which includes: optimum photometric and background apertures, effective area curves, colour transformations, conversion factors for count rates to flux, and the photometric zero points (which are accurate to better than 4 per cent) for each of the seven uvot broadband filters. the calibration was performed with observations of standard stars and standard star fields that represent a wide range of spectral star types. the calibration results include the position dependent uniformity, and instrument response over the 1600-8000a operational range. because the uvot is a photon counting instrument, we also discuss the effect of coincidence loss on the calibration results. we provide practical guidelines for using the calibration in uvot data analysis. the results presented here supersede previous calibration results.',
   'qas': [{'question': 'Title',
     

In [9]:
len(output['data'])

3094

In [10]:
# Convert test data

output = {}
output['version'] = 'v1.0'
output['data'] = []

for line in test:
    paragraphs = []
    
    context = line[0]
    
    qas = []
    question = 'Title'
    qid = line[1]
    if type(context) != str or type(question) != str:
        print(context, type(context))
        print(answer, type(answer))
        print(question, type(question))
        continue
    answers = []
    answers.append({'answer_start': 1000000, 'text': '__None__'})  #'answer_start': 1000000, 
    qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
    
    paragraphs.append({'context': context, 'qas': qas})
    output['data'].append({'title': 'None', 'paragraphs': paragraphs})

with open('data/test.json', 'w') as outfile:
    json.dump(output, outfile)

In [11]:
!cd /kaggle/input/pytorchtransformers/transformers-2.5.1; pip install .

Processing /kaggle/input/pytorchtransformers/transformers-2.5.1
  Created wheel for transformers: filename=transformers-2.5.1-py3-none-any.whl size=499131 sha256=7083d54c81ab64d32623c9e56d27576fa2dfdb28da9e04b10500ada7d2bb4c15
  Stored in directory: /root/.cache/pip/wheels/b0/58/32/c9de6e928489e1884c4a65f0aaf8a6ebe5484617eaadbea3b6
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 2.5.1
    Uninstalling transformers-2.5.1:
      Successfully uninstalled transformers-2.5.1


In [12]:
!mkdir results_roberta_large

In [13]:
!python /kaggle/input/pytorchtransformers/transformers-2.5.1/examples/run_squad.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --do_train \
    --do_eval \
    --do_lower_case \
    --data_dir ./data \
    --train_file train.json \
    --predict_file test.json \
    --learning_rate 3e-5 \
    --num_train_epochs 1 \
    --max_seq_length 384 \
    --doc_stride 128 \
    --output_dir results_roberta_large \
    --per_gpu_eval_batch_size=4  \
    --per_gpu_train_batch_size=4   \
    --save_steps 5000

2020-07-29 13:37:54.967817: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100%|██████████████████████████████| 433/433 [00:00<00:00, 285kB/s]
Downloading: 100%|███████████████████████████| 232k/232k [00:00<00:00, 3.86MB/s]
Downloading: 100%|███████████████████████████| 440M/440M [00:11<00:00, 39.0MB/s]
Iteration:  11%|███▍                           | 88/783 [00:18<02:19,  4.99it/s]


Iteration:  24%|███████                       | 185/783 [00:38<01:59,  4.99it/s]


Iteration:  36%|██████████▊                   | 282/783 [00:58<01:40,  4.98it/s]


Iteration:  48%|██████████████▌               | 379/783 [01:17<01:20,  5.02it/s]


Iteration:  61%|██████████████████▏           | 476/783 [01:37<01:01,  5.00it/s]


Iteration:  64%|███████████████████           | 498/783 [01:42<01:08,  4.17it/s]/opt/conda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "

Iteration:  73%|█████████████████████▊        | 570/783 [01:56<00:42,  5.02it/s]


Iteration:  85%|█████████████████████████▌    | 667/783 [02:16<00:23,  5.00it/s]


Iteration:  98%|█████████████████████████████▎| 764/783 [02:35<00:04,  4.69it/s]


Evaluating: 100%|█████████████████████████████| 256/256 [00:14<00:00, 17.85it/s]


In [14]:
train = pd.read_csv('/kaggle/input/title-generation/train.csv')
test = pd.read_csv('/kaggle/input/title-generation/test.csv')

In [15]:
train_abstracts = train['abstract'].str.lower()
test_abstracts = test['abstract'].str.lower()

In [16]:
#в тестовых данных содержаться часть трейна, подставим эти данные в ответ
duplicate_ids = {}

for i, abstract in tqdm(enumerate(test_abstracts)):
    inclusion_series = (abstract == train_abstracts)
    if inclusion_series.sum():
        test_id = i
        train_id = inclusion_series.idxmax()
        duplicate_ids[test_id] =  train_id
        
len(duplicate_ids)

431

In [17]:
# Copy predictions to submission file.
predictions = json.load(open('results_roberta_large/predictions_.json', 'r'))
submission = pd.read_csv(open('/kaggle/input/title-generation/sample_submission.csv', 'r'))
submission_df = pd.read_csv(open('/kaggle/input/title-generation/sample_submission.csv', 'r')).iloc[:1000]
for test_id, train_id in duplicate_ids.items():
    predictions[str(test_id)] = train.loc[train_id, 'title']
for i in range(len(predictions)):
    id_ = submission_df['Id'][i] 
    submission_df.loc[i, 'title'] = predictions[str(id_)]
    
    
submission_df.to_csv('predicted_titles.csv', index=False)

In [21]:
#получилось вполне человекоподобно
submission_df['title'].head()

0                    a novel simple neural network
1                                          doc2vec
2    a two-way LSTM with conditional random fields
3                               coverage-based NMT
4                   unsupervised image translation
Name: title, dtype: object